<a href="https://colab.research.google.com/github/ansharyis/ml-colab-project/blob/main/notebooks/03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DATA_DIR = "/content/drive/MyDrive/ML_Project_Data"


In [ ]:
# Project: ML Weight Prediction
# Notebook: ....
# Owner: ....
# Description: ....

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline